This is 3des+tent

In [ ]:
import time
import secrets
from base64 import b64encode, b64decode
from Crypto.Cipher import DES3
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad, unpad

# Baudot encoding for a-z and space only (from your image)
baudot = {
    ' ': '00000', 't': '00001', 'o': '00011', 'h': '00101', 'n': '00110',
    'm': '00111', 'l': '01001', 'r': '01010', 'g': '01011', 'i': '01100',
    'p': '01101', 'c': '01110', 'v': '01111', 'e': '10000', 'z': '10001',
    'd': '10010', 'b': '10011', 's': '10100', 'y': '10101', 'f': '10110',
    'x': '10111', 'a': '11000', 'w': '11001', 'j': '11010', 'u': '11100',
    'q': '11101', 'k': '11110'
}
reverse_baudot = {v: k for k, v in baudot.items()}

def to_baudot_bits(text):
    return ''.join([baudot[c] for c in text.lower() if c in baudot])

def bits_to_bytes(bits):
    while len(bits) % 8 != 0:
        bits += '0'
    return [int(bits[i:i+8], 2) for i in range(0, len(bits), 8)]

def bytes_to_bits(byte_list):
    return ''.join([format(b, '08b') for b in byte_list])

def from_baudot_bits(bits):
    return ''.join([reverse_baudot.get(bits[i:i+5], '?') for i in range(0, len(bits), 5)])

# === 🆕 TENT MAP instead of logistic ===
def tent_map(x0, r, length):
    x = x0
    chaos = []
    for _ in range(length):
        if x < 0.5:
            x = r * x
        else:
            x = r * (1 - x)
        chaos.append(int(x * 256) % 256)
    return chaos
# =======================================

def xor_lists(a, b):
    return [x ^ y for x, y in zip(a, b)]

# === 3DES ENCRYPTION ===
def des3_encrypt(byte_list, key):
    key = SHA256.new(key).digest()[:24]
    iv = secrets.token_bytes(8)
    cipher = DES3.new(key, DES3.MODE_CBC, iv)
    padded = pad(bytes(byte_list), 8)
    ciphertext = cipher.encrypt(padded)
    return b64encode(iv + ciphertext).decode()

def des3_decrypt(data_b64, key):
    key = SHA256.new(key).digest()[:24]
    raw = b64decode(data_b64)
    iv, ciphertext = raw[:8], raw[8:]
    cipher = DES3.new(key, DES3.MODE_CBC, iv)
    decrypted = unpad(cipher.decrypt(ciphertext), 8)
    return list(decrypted)
# ========================

def encrypt(text, key, x0=0.7, r=1.99):
    print(f"\n🔐 ENCRYPTING TEXT: {text[:50]}...")
    bits = to_baudot_bits(text)
    print(f"Baudot Bits: {bits[:80]}...")
    baudot_bytes = bits_to_bytes(bits)
    print(f"Baudot Bytes: {baudot_bytes[:10]}...")
    chaos = tent_map(x0, r, len(baudot_bytes))
    print(f"Tent Chaotic Bytes: {chaos[:10]}...")
    xored = xor_lists(baudot_bytes, chaos)
    print(f"XORed Bytes: {xored[:10]}...")
    des3_enc = des3_encrypt(xored, key)
    print(f"🔒 3DES Encrypted (Base64): {des3_enc[:80]}...")
    print(f"🧮 Baudot Bit Length: {len(bits)} (save this for decryption)")
    return des3_enc, len(bits)

def decrypt(des3_enc, key, bit_len, x0=0.7, r=1.99):
    print("\n🔓 DECRYPTING...")
    xored = des3_decrypt(des3_enc, key)
    print(f"Decrypted XORed Bytes: {xored[:10]}...")
    chaos = tent_map(x0, r, len(xored))
    print(f"Regenerated Tent Bytes: {chaos[:10]}...")
    baudot_bytes = xor_lists(xored, chaos)
    print(f"Recovered Baudot Bytes: {baudot_bytes[:10]}...")
    bits = bytes_to_bits(baudot_bytes)[:bit_len]
    print(f"Baudot Bits: {bits[:80]}...")
    text = from_baudot_bits(bits).upper()
    print(f"✅ Decrypted Text Preview: {text[:80]}...")
    return text

def generate_random_key(length=24):
    return secrets.token_bytes(length)

def main():
    file_path = "/content/text_200kb.txt"
    with open(file_path, 'r') as f:
        raw_text = f.read().lower()

    # Clean input to valid Baudot characters
    text = ''.join(c for c in raw_text if c in baudot)

    des3_key = generate_random_key()
    print(f"\n🗝 Random 3DES Key Used (Hex): {des3_key.hex()}")

    # ENCRYPTION
    start_enc = time.time()
    des3_enc, bit_len = encrypt(text, des3_key)
    end_enc = time.time()

    # DECRYPTION
    start_dec = time.time()
    decrypted_text = decrypt(des3_enc, des3_key, bit_len)
    end_dec = time.time()

    print(f"\n⏱ Encryption Time: {(end_enc - start_enc) * 1000:.2f} ms")
    print(f"⏱ Decryption Time: {(end_dec - start_dec) * 1000:.2f} ms")

if _name_ == "_main_":
    main()

In [ ]:
this is 3 des with cosine polynomial

In [ ]:
import time
import secrets
import math
from base64 import b64encode, b64decode
from Crypto.Cipher import DES3
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad, unpad

# Baudot encoding for a-z and space only (from your image)
baudot = {
    ' ': '00000', 't': '00001', 'o': '00011', 'h': '00101', 'n': '00110',
    'm': '00111', 'l': '01001', 'r': '01010', 'g': '01011', 'i': '01100',
    'p': '01101', 'c': '01110', 'v': '01111', 'e': '10000', 'z': '10001',
    'd': '10010', 'b': '10011', 's': '10100', 'y': '10101', 'f': '10110',
    'x': '10111', 'a': '11000', 'w': '11001', 'j': '11010', 'u': '11100',
    'q': '11101', 'k': '11110'
}
reverse_baudot = {v: k for k, v in baudot.items()}

def to_baudot_bits(text):
    return ''.join([baudot[c] for c in text.lower() if c in baudot])

def bits_to_bytes(bits):
    while len(bits) % 8 != 0:
        bits += '0'
    return [int(bits[i:i+8], 2) for i in range(0, len(bits), 8)]

def bytes_to_bits(byte_list):
    return ''.join([format(b, '08b') for b in byte_list])

def from_baudot_bits(bits):
    return ''.join([reverse_baudot.get(bits[i:i+5], '?') for i in range(0, len(bits), 5)])

# === ✅ CORRECT COSINE POLYNOMIAL MAP ===
def cosine_polynomial_map(x0, delta, length):
    x = x0
    chaos = []
    for _ in range(length):
        x = math.cos(delta * x**3 + x)
        x_norm = (x + 1) / 2  # Normalize from [-1, 1] to [0, 1]
        chaos.append(int(x_norm * 256) % 256)
    return chaos
# ========================================

def xor_lists(a, b):
    return [x ^ y for x, y in zip(a, b)]

# === 3DES ENCRYPTION ===
def des3_encrypt(byte_list, key):
    key = SHA256.new(key).digest()[:24]
    iv = secrets.token_bytes(8)
    cipher = DES3.new(key, DES3.MODE_CBC, iv)
    padded = pad(bytes(byte_list), 8)
    ciphertext = cipher.encrypt(padded)
    return b64encode(iv + ciphertext).decode()

def des3_decrypt(data_b64, key):
    key = SHA256.new(key).digest()[:24]
    raw = b64decode(data_b64)
    iv, ciphertext = raw[:8], raw[8:]
    cipher = DES3.new(key, DES3.MODE_CBC, iv)
    decrypted = unpad(cipher.decrypt(ciphertext), 8)
    return list(decrypted)
# ========================

def encrypt(text, key, x0=0.7, delta=1.2):
    print(f"\n🔐 ENCRYPTING TEXT: {text[:50]}...")
    bits = to_baudot_bits(text)
    print(f"Baudot Bits: {bits[:80]}...")
    baudot_bytes = bits_to_bytes(bits)
    print(f"Baudot Bytes: {baudot_bytes[:10]}...")
    chaos = cosine_polynomial_map(x0, delta, len(baudot_bytes))
    print(f"Cosine Chaotic Bytes: {chaos[:10]}...")
    xored = xor_lists(baudot_bytes, chaos)
    print(f"XORed Bytes: {xored[:10]}...")
    des3_enc = des3_encrypt(xored, key)
    print(f"🔒 3DES Encrypted (Base64): {des3_enc[:80]}...")
    print(f"🧮 Baudot Bit Length: {len(bits)} (save this for decryption)")
    return des3_enc, len(bits)

def decrypt(des3_enc, key, bit_len, x0=0.7, delta=1.2):
    print("\n🔓 DECRYPTING...")
    xored = des3_decrypt(des3_enc, key)
    print(f"Decrypted XORed Bytes: {xored[:10]}...")
    chaos = cosine_polynomial_map(x0, delta, len(xored))
    print(f"Regenerated Cosine Bytes: {chaos[:10]}...")
    baudot_bytes = xor_lists(xored, chaos)
    print(f"Recovered Baudot Bytes: {baudot_bytes[:10]}...")
    bits = bytes_to_bits(baudot_bytes)[:bit_len]
    print(f"Baudot Bits: {bits[:80]}...")
    text = from_baudot_bits(bits).upper()
    print(f"✅ Decrypted Text Preview: {text[:80]}...")
    return text

def generate_random_key(length=24):
    return secrets.token_bytes(length)

def main():
    file_path = "/content/text_200kb.txt"
    with open(file_path, 'r') as f:
        raw_text = f.read().lower()

    # Clean input to valid Baudot characters
    text = ''.join(c for c in raw_text if c in baudot)

    des3_key = generate_random_key()
    print(f"\n🗝 Random 3DES Key Used (Hex): {des3_key.hex()}")

    # ENCRYPTION
    start_enc = time.time()
    des3_enc, bit_len = encrypt(text, des3_key)
    end_enc = time.time()

    # DECRYPTION
    start_dec = time.time()
    decrypted_text = decrypt(des3_enc, des3_key, bit_len)
    end_dec = time.time()

    print(f"\n⏱ Encryption Time: {(end_enc - start_enc) * 1000:.2f} ms")
    print(f"⏱ Decryption Time: {(end_dec - start_dec) * 1000:.2f} ms")

if _name_ == "_main_":
    main()

3des+logistic

In [ ]:
import time
import secrets
from base64 import b64encode, b64decode
from Crypto.Cipher import DES3
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad, unpad

# Baudot encoding for a-z and space only (from your image)
baudot = {
    ' ': '00000', 't': '00001', 'o': '00011', 'h': '00101', 'n': '00110',
    'm': '00111', 'l': '01001', 'r': '01010', 'g': '01011', 'i': '01100',
    'p': '01101', 'c': '01110', 'v': '01111', 'e': '10000', 'z': '10001',
    'd': '10010', 'b': '10011', 's': '10100', 'y': '10101', 'f': '10110',
    'x': '10111', 'a': '11000', 'w': '11001', 'j': '11010', 'u': '11100',
    'q': '11101', 'k': '11110'
}
reverse_baudot = {v: k for k, v in baudot.items()}

def to_baudot_bits(text):
    return ''.join([baudot[c] for c in text.lower() if c in baudot])

def bits_to_bytes(bits):
    while len(bits) % 8 != 0:
        bits += '0'
    return [int(bits[i:i+8], 2) for i in range(0, len(bits), 8)]

def bytes_to_bits(byte_list):
    return ''.join([format(b, '08b') for b in byte_list])

def from_baudot_bits(bits):
    return ''.join([reverse_baudot.get(bits[i:i+5], '?') for i in range(0, len(bits), 5)])

def logistic_map(x0, delta, length):
    x = x0
    chaos = []
    for _ in range(length):
        x = delta * x * (1 - x)
        chaos.append(int(x * 256) % 256)
    return chaos

def xor_lists(a, b):
    return [x ^ y for x, y in zip(a, b)]

# === 3DES ENCRYPTION ===
def des3_encrypt(byte_list, key):
    key = SHA256.new(key).digest()[:24]
    iv = secrets.token_bytes(8)
    cipher = DES3.new(key, DES3.MODE_CBC, iv)
    padded = pad(bytes(byte_list), 8)
    ciphertext = cipher.encrypt(padded)
    return b64encode(iv + ciphertext).decode()

def des3_decrypt(data_b64, key):
    key = SHA256.new(key).digest()[:24]
    raw = b64decode(data_b64)
    iv, ciphertext = raw[:8], raw[8:]
    cipher = DES3.new(key, DES3.MODE_CBC, iv)
    decrypted = unpad(cipher.decrypt(ciphertext), 8)
    return list(decrypted)
# ========================

def encrypt(text, key, x0=0.7, delta=3.9):
    print(f"\n🔐 ENCRYPTING TEXT: {text[:50]}...")
    bits = to_baudot_bits(text)
    print(f"Baudot Bits: {bits[:80]}...")
    baudot_bytes = bits_to_bytes(bits)
    print(f"Baudot Bytes: {baudot_bytes[:10]}...")
    chaos = logistic_map(x0, delta, len(baudot_bytes))
    print(f"Logistic Chaotic Bytes: {chaos[:10]}...")
    xored = xor_lists(baudot_bytes, chaos)
    print(f"XORed Bytes: {xored[:10]}...")
    des3_enc = des3_encrypt(xored, key)
    print(f"🔒 3DES Encrypted (Base64): {des3_enc[:80]}...")
    print(f"🧮 Baudot Bit Length: {len(bits)} (save this for decryption)")
    return des3_enc, len(bits)

def decrypt(des3_enc, key, bit_len, x0=0.7, delta=3.9):
    print("\n🔓 DECRYPTING...")
    xored = des3_decrypt(des3_enc, key)
    print(f"Decrypted XORed Bytes: {xored[:10]}...")
    chaos = logistic_map(x0, delta, len(xored))
    print(f"Regenerated Chaotic Bytes: {chaos[:10]}...")
    baudot_bytes = xor_lists(xored, chaos)
    print(f"Recovered Baudot Bytes: {baudot_bytes[:10]}...")
    bits = bytes_to_bits(baudot_bytes)[:bit_len]
    print(f"Baudot Bits: {bits[:80]}...")
    text = from_baudot_bits(bits).upper()
    print(f"✅ Decrypted Text Preview: {text[:80]}...")
    return text

def generate_random_key(length=24):
    return secrets.token_bytes(length)

def main():
    file_path = "/content/text_200kb.txt"
    with open(file_path, 'r') as f:
        raw_text = f.read().lower()

    # Clean input to valid Baudot characters
    text = ''.join(c for c in raw_text if c in baudot)

    des3_key = generate_random_key()
    print(f"\n🗝 Random 3DES Key Used (Hex): {des3_key.hex()}")

    # ENCRYPTION
    start_enc = time.time()
    des3_enc, bit_len = encrypt(text, des3_key)
    end_enc = time.time()

    # DECRYPTION
    start_dec = time.time()
    decrypted_text = decrypt(des3_enc, des3_key, bit_len)
    end_dec = time.time()

    print(f"\n⏱ Encryption Time: {(end_enc - start_enc) * 1000:.2f} ms")
    print(f"⏱ Decryption Time: {(end_dec - start_dec) * 1000:.2f} ms")

if _name_ == "_main_":
    main()